In [2]:
import gym
import gym_watten
from gym_watten.envs.watten_env import Color, Value
import numpy as np
from time import sleep
import random
from tqdm import tqdm

In [3]:
env = gym.make("Watten-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_watten.envs.watten_env.WattenEnv'>' has deprecated methods. Compatibility code invoked.


# Search

In [4]:
def search(obs):
    global sample_outputs, sample_inputs, next_index
    state = env.get_state()
    n = len(env.players[env.current_player].hand_cards)   
    p = 0
    local_index = next_index
    next_index += 1
    sample_inputs[0][local_index] = np.array(obs[0])
    sample_inputs[1][local_index] = np.array(obs[1])
    current_player = env.current_player
    
    for i in range(n):
        card_id = env.players[env.current_player].hand_cards[i].id
        obs, rew, is_done, _ = env.step(card_id)
        
        if is_done:
            sample_outputs[local_index][card_id] = (rew[0] > 0)
        else:
            sample_outputs[local_index][card_id] = ((1 - search(obs)) if current_player != env.current_player else search(obs))
        env.set_state(state)
        p += sample_outputs[local_index][card_id]
        
    return sample_outputs[local_index].max()

In [5]:
def reset_samples(iterations):
    global sample_outputs, sample_inputs,next_index
    sample_outputs = [np.zeros((30000*iterations, 32), dtype=float), np.zeros((30000*iterations, 1), dtype=int)]
    sample_inputs = [np.zeros((30000*iterations, 4, 8, 2), dtype=int), np.zeros((30000*iterations, 4), dtype=int)]
    next_index = 0
    
def postprocess_samples():
    global sample_outputs, sample_inputs,next_index
    for i in range(len(sample_inputs)):
        sample_inputs[i] = sample_inputs[i][:next_index]
    for i in range(len(sample_outputs)):
        sample_outputs[i] = sample_outputs[i][:next_index]
    
    #exp = np.exp(sample_outputs)
    #exp[np.where(sample_outputs == -1)] = 0
    #sample_outputs = exp / np.expand_dims(np.sum(exp, axis=-1), -1)
    
def generate_training_data(iterations=1):
    global sample_outputs, sample_inputs,next_index
    reset_samples(iterations)
    
    for i in range(iterations):
        obs = env.reset()
        search(obs)    
        
    postprocess_samples()  
    return sample_inputs, sample_outputs

# MCTS

In [11]:
class State:
    def __init__(self, p, env_state, end_v=0):
        self.childs = []
        self.n = 0
        self.w = 0
        self.p = p
        self.env_state = env_state
        self.end_v = end_v
    
    def is_leaf_node(self):
        return len(self.childs) == 0
    
    def size(self):
        size = 1
        for child in self.childs:
            size += child.size()
        return size
    
def mcts_sample(state, model):
    
    if state.is_leaf_node():
        if state.end_v != 0:
            return state.end_v
        else:
            env.set_state(state.env_state)

            p, v = model.predict_single(env.regenerate_obs())

            hand_cards = env.players[env.current_player].hand_cards[:]        
            for card in hand_cards:
                obs, rew, is_done, _ = env.step(card.id)

                new_state = State(p[card.id], env.get_state(), 0 if not is_done else (1 if rew[0] > 0 else -1))
                state.childs.append(new_state)

                env.set_state(state.env_state)

            return v[0]  
    else:
        n_sum = 0
        for child in state.childs:
            n_sum += child.n
        
        max_u = 0
        max_child = None
        for child in state.childs:
            if child.n > 0:
                u = child.w / child.n 
            else:
                u = 0
            
            u += child.p * np.sqrt(n_sum) / (1 + child.n)

            if max_child is None or u > max_u:
                max_u, max_child = u, child

        v = mcts_sample(max_child, model)
        max_child.w += v
        max_child.n += 1
        return v

def mcts_game_step(root):
    for i in range(50):
        mcts_sample(root, model)
        
    p = [child.n for child in root.childs]
    p /= np.sum(p)
    return np.random.choice(np.arange(0, len(p)), p=p), p
    
def mcts_game():
    global sample_inputs, sample_outputs, next_index
    obs = env.reset()
    root = State(1, env.get_state())
    is_done = False
    start_index = next_index
    
    while not is_done:
        game_state = env.get_state()
        a, p = mcts_game_step(root)
        env.set_state(game_state)
        
        sample_inputs[0][next_index] = np.array(obs[0])
        sample_inputs[1][next_index] = np.array(obs[1])
        for i, card in enumerate(env.players[env.current_player].hand_cards):
            sample_outputs[0][next_index][card.id] = p[i]    
        sample_outputs[1][next_index] = 1 if env.current_player is 0 else -1
        
        next_index += 1
        
        obs, rew, is_done, _ = env.step(env.players[env.current_player].hand_cards[a].id)
        root = root.childs[a]
        
    for i in range(start_index, next_index):
        sample_outputs[1][i][0] *= (1 if env.current_player is 0 else -1) * (1 if rew[0] > 0 else -1)
        
def mcts_generate():
    reset_samples(1)
    for i in tqdm(range(30)):
        mcts_game()
    postprocess_samples()

In [12]:
mcts_generate()

100%|██████████| 30/30 [00:05<00:00,  5.39it/s]


In [13]:
next_index

252

In [143]:
[i for i in range(10, 20)]

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [92]:
env.set_state(state_org)

print([state.n for state in root.childs])
print([state.w for state in root.childs])
root.size()

[311, 277, 351, 362, 298]
[11.83459559478797, 7.551910203765146, 15.525370486284373, 17.178428841376444, 8.31264715976431]


4845

# Play

In [17]:
def match(agents, render=False):
    obs, is_done = env.reset(), False    
    
    while not is_done:
        prediction = agents[env.current_player].predict_single(obs)[0]
        
        valid_moves = np.array([1 if card in env.players[env.current_player].hand_cards else 0 for card in env.cards])       
        prediction_valid = np.ma.masked_where(valid_moves == 0, prediction)
        
        obs, rew, is_done, _ = env.step(np.argmax(prediction_valid))
        
        if render:
            env.render('human')
            sleep(1)
        
    return env.current_player

def compare(agent1, agent2):
    agents = [agent1, agent2]
    first_player_wins = 0
    
    for i in range(1000):
        start_player = random.randint(0, 1)
        winner = match([agents[start_player], agents[1 - start_player]])
        first_player_wins += ((winner == 0) == (start_player == 0))
    
    return first_player_wins / 1000      

In [19]:
compare(model, model)

0.512

# Train

In [8]:
from keras.models import Sequential, clone_model
from keras.layers import Dense, Activation, Input, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, Merge, Flatten, BatchNormalization, add
from keras.layers.merge import concatenate
from keras.models import Model

import keras.backend as K
from IPython.display import clear_output
from keras import optimizers
import math

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
class ImprovedModel(Model):
    def __init__(self, inputs, outputs):
        super().__init__(inputs, outputs)

    def predict_single(self, inputs):
        inputs = [np.expand_dims(input, 0) for input in inputs]
        outputs = model.predict(inputs)
        return [output[0] for output in outputs]

def mean_pred(y_true, y_pred):
    return K.mean(K.equal(K.gather(y_true, K.argmax(y_true, axis=-1)), K.gather(y_true, K.argmax(y_pred, axis=-1))))


def residual_layer(input_block, filters, kernel_size):

    x = conv_layer(input_block, filters, kernel_size)

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)

    x = add([input_block, x])

    x = LeakyReLU()(x)

    return (x)

def conv_layer(x, filters, kernel_size):

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)
    x = LeakyReLU()(x)

    return (x)

def build_model():
    input_1 = Input((4,8,2))
    convnet = input_1

    convnet = conv_layer(convnet, 32, (3, 3))
    convnet = residual_layer(convnet, 32, (3, 3))
    convnet = residual_layer(convnet, 32, (3, 3))
    convnet = residual_layer(convnet, 32, (3, 3))
    convnet = conv_layer(convnet, 2, (1, 1))
    #convnet = Conv2D(32, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    convnet = Flatten()(convnet)

    input_2 = Input((4,))

    policy_out = concatenate([convnet, input_2])
    #dense_out = Dense(128, activation='relu')(dense_out)
    #dense_out = Dense(256, activation='relu')(dense_out)
    #policy_out = Dense(256, activation='relu')(dense_out)
    policy_out = Dense(32, activation='sigmoid')(policy_out)

    value_out = concatenate([convnet, input_2])
    #dense_out = Dense(128, activation='relu')(dense_out)
    #dense_out = Dense(256, activation='relu')(dense_out)
    value_out = Dense(256, activation='relu')(value_out)
    value_out = Dense(1, activation='tanh')(value_out)

    model = ImprovedModel(inputs=[input_1, input_2], outputs=[policy_out, value_out])

    adam = optimizers.Adam()
    model.compile(optimizer=adam,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    return model

In [24]:
model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 4, 8, 2)      0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 4, 8, 32)     576         input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, 4, 8, 32)     0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 4, 8, 32)     9216        leaky_re_lu_9[0][0]              
__________________________________________________________________________________________________
leaky_re_l

In [27]:
prev_states = []
for i in range(10):
    prev_states.append(build_model())
    prev_states[-1].set_weights(model.get_weights())
    #enerate_training_data(100)    
    mcts_generate()
    p = np.random.permutation(int(len(sample_outputs[0]) * 0.05))
    for i in range(len(sample_outputs)):
        sample_outputs[i] = sample_outputs[i][p]
    for i in range(len(sample_inputs)):
        sample_inputs[i] = sample_inputs[i][p]
        
    # Train the model, iterating on the data in batches of 32 samples
    model.fit(sample_inputs, sample_outputs, epochs=1, batch_size=64)    
    
    print("New model wins " + str([compare(model, prev_state) * 100 for prev_state in prev_states]) + "%!")

100%|██████████| 30/30 [00:05<00:00,  5.09it/s]

Epoch 1/1
13/13 [==============================] - 0s 873us/step - loss: 1.2021 - dense_4_loss: 0.2286 - dense_6_loss: 0.9735 - dense_4_acc: 0.0769 - dense_6_acc: 0.0000e+00


New model wins [47.4]%!


100%|██████████| 30/30 [00:05<00:00,  5.08it/s]

Epoch 1/1
13/13 [==============================] - 0s 821us/step - loss: 1.2485 - dense_4_loss: 0.2331 - dense_6_loss: 1.0154 - dense_4_acc: 0.0769 - dense_6_acc: 0.0000e+00


...

Epoch 1/1
13/13 [==============================] - 0s 1ms/step - loss: 1.2686 - dense_4_loss: 0.2231 - dense_6_loss: 1.0455 - dense_4_acc: 0.0769 - dense_6_acc: 0.0000e+00


New model wins [53.800000000000004, 50.0, 49.5, 48.699999999999996, 48.1, 49.4, 52.2, 48.5, 49.0]%!


100%|██████████| 30/30 [00:07<00:00,  3.85it/s]

Epoch 1/1
13/13 [==============================] - 0s 1ms/step - loss: 1.1975 - dense_4_loss: 0.2272 - dense_6_loss: 0.9703 - dense_4_acc: 0.0000e+00 - dense_6_acc: 0.0000e+00


New model wins [50.5, 49.0, 50.0, 48.199999999999996, 47.8, 48.199999999999996, 50.1, 51.2, 50.4, 49.7]%!


In [30]:
model.save('model.h5')

In [ ]:
model.load_weights('model.h5')

# Eval

In [ ]:
generate_training_data(100)    
p = np.random.permutation(len(sample_inputs))
sample_inputs, sample_outputs = sample_inputs[p], sample_outputs[p]

In [ ]:
model.evaluate(sample_inputs, sample_outputs)

# Predict

In [12]:
def generate_input(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    obs = [np.zeros([4, 8, 2]), np.zeros([4])]
    
    for card in hand_cards:
        obs[0][card.color.value][card.value.value][0] = 1
        
    if table_card is not None:
        obs[0][table_card.color.value][table_card.value.value][1] = 1

    obs[1][0] = (own_tricks == 1 or own_tricks == 3)
    obs[1][1] = (own_tricks == 2 or own_tricks == 3)

    obs[1][2] = (opponent_tricks == 1 or opponent_tricks == 3)
    obs[1][3] = (opponent_tricks == 2 or opponent_tricks == 3)
    
    return [np.expand_dims(obs[0], 0), np.expand_dims(obs[1], 0)]

In [13]:
def card(color, value):
    for card in env.cards:
        if card.value == value and card.color == color:
            return card
        
    return None

In [14]:
def calc_correct_output_sample(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    global sample_outputs
    env.reset()
    env.cards_left = env.cards[:]
    random.shuffle(env.cards_left)
    
    for card in hand_cards:
        env.cards_left.remove(card)
    env.players[0].hand_cards = hand_cards[:]
        
    env.players[1].hand_cards = []
    for i in range(len(hand_cards) - (1 if table_card is not None else 0)):
        env.players[1].hand_cards.append(env.cards_left.pop())       
    
    env.players[0].tricks = own_tricks
    env.players[1].tricks = opponent_tricks
    env.table_card = table_card
    
    #obs, rew, is_done, _ = env.step(hand_cards[0].id)
   # print(rew, is_done)

    #env.render('human')
    reset_samples(1)
    search(env.regenerate_obs())
    postprocess_samples()
        
    return sample_outputs[0]

In [15]:
def calc_correct_output(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):    
    correct_output = None
    for i in range(100):
        sample_outputs = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
        if correct_output is None:
            correct_output = sample_outputs
        else:
            correct_output += sample_outputs
    return correct_output / 100

In [16]:
def predict(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
    output = model.predict(input)[0]
    correct_output = calc_correct_output(hand_cards, table_card, own_tricks, opponent_tricks)
    print(output)
    for card in hand_cards:
        print(str(card.color) + " " + str(card.value) + " => " + str(output[card.id]) + " (" + str(correct_output[card.id]) + ")")

In [17]:
def eval():
    correct = 0
    for i in range(100):
        own_tricks = random.randint(0, 2)
        opponent_tricks = random.randint(0, 2)
        table_card_enabled = random.randint(0, 1)
        hand_cards = random.sample(env.cards, 5 - own_tricks - opponent_tricks + table_card_enabled)
        table_card = hand_cards.pop() if table_card_enabled else None

        input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
        output = model.predict(input)[0]
        correct_output = calc_correct_output(hand_cards, table_card, own_tricks, opponent_tricks)
        
        #print(output, np.argmax(output)) 
        #print(correct_output, np.argmax(correct_output))
        correct += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        if correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
            print(correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)])
    return correct / 100

In [ ]:
eval()

In [39]:
predict([
    card(Color.EICHEL, Value.ACHT), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ], None, 0, 2)


[1.38462798e-04 1.04060797e-02 5.30510169e-06 8.09911853e-06
 1.11277751e-03 2.58240470e-04 1.98464707e-01 5.46842566e-05
 1.13082235e-04 6.16233214e-04 2.23065595e-04 6.11462610e-06
 4.77710346e-05 5.13106759e-04 2.47087708e-04 3.29982169e-04
 5.98560274e-01 6.36516870e-06 3.55967604e-05 2.20201309e-06
 6.82708696e-06 4.63786448e-04 3.89412162e-06 6.65397791e-04
 1.52922337e-06 3.25583853e-04 3.94351773e-05 1.06607535e-04
 1.63332428e-04 1.17983063e-05 1.69128471e-04 1.86277484e-05]
Color.EICHEL Value.ACHT => 0.1984647 (0.16)
Color.EICHEL Value.KOENIG => 0.01040608 (0.16)
Color.HERZ Value.SAU => 0.5985603 (0.16)


In [40]:
predict([
    card(Color.EICHEL, Value.SAU), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ],card(Color.EICHEL, Value.ZEHN), 1, 0)

[8.5775316e-02 1.3446440e-01 3.0592244e-05 5.1846814e-06 6.2620740e-05
 3.8401669e-04 1.3691410e-04 8.2836683e-05 7.3889018e-06 1.3270145e-05
 4.0721407e-05 1.3476485e-06 5.4208715e-05 1.9332780e-05 4.7947484e-05
 5.3705394e-06 9.5115882e-01 1.5323345e-05 4.7273261e-05 1.3709297e-06
 3.6681984e-05 6.0819509e-04 4.7694807e-04 4.3128894e-05 3.2106681e-07
 1.6626344e-03 3.0777740e-04 2.0046342e-05 2.0370177e-05 4.6278772e-04
 4.4946846e-06 6.6884668e-06]
Color.EICHEL Value.SAU => 0.085775316 (0.98)
Color.EICHEL Value.KOENIG => 0.1344644 (0.98)
Color.HERZ Value.SAU => 0.9511588 (0.57)
